# CryptoHack_Mathematics

This category covers more advanced cryptographic math. It's not necessary to solve these before moving to the Block Cipher and RSA sections. However, the first challenges will expand your modular toolbox, while the later ones are reported to be among the most satisfying puzzles to solve on CryptoHack. There are 22 challenges.

# Modular Math
## Quadratic Residues

We've looked at multiplication and division in modular arithmetic, but what does it mean to take the square root modulo an integer?

For the following discussion, let's work modulo $p = 29$. We can take the integer $a = 11$ and calculate $a^2 = 5\ mod\ 29$.

As $a = 11$, $a^2 = 5$, we say the square root of 5 is 11.

This feels good, but now let's think about the square root of 18. From the above, we know we need to find some integer a such that $a^2 = 18$

Your first idea might be to start with $a = 1$ and loop to $a = p-1$. In this discussion p isn't too large and we can quickly look.

Have a go, try coding this and see what you find. If you've coded it right, you'll find that for all $a ∈ F_p^*$ you never find an a such that $a^2 = 18$.

What we are seeing, is that for the elements of $F_p^*$, not every element has a square root. In fact, what we find is that for roughly one half of the elements of $F_p^*$, there is no square root.

We say that an integer $x$ is a Quadratic Residue if there exists an $a$ such that $a^2 = x\ mod\ p$. If there is no such solution, then the integer is a Quadratic Non-Residue.

In other words, $x$ is a quadratic residue when it is possible to take the square root of $x$ modulo an integer $p$.

In the below list there are two non-quadratic residues and one quadratic residue.

Find the quadratic residue and then calculate its square root. Of the two possible roots, submit the smaller one as the flag.

If $a^2 = x$ then $(-a)^2 = x$. So if $x$ is a quadratic residue in some finite field, then there are always two solutions for $a$.

```
p = 29
ints = [14, 6, 11] 
```

In [5]:
squares=[(x,x**2%29) for x in range(29)]
for item in squares:
    if item[1]==6:
        print(item)

(8, 6)
(21, 6)


So the smaller quadratic residue is 8. On to the next one!
## Legendre Symbol

In Quadratic Residues we learnt what it means to take the square root modulo an integer. We also saw that taking a root isn't always possible.

In the previous case when $p = 29$, even the simplest method of calculating the square root was fast enough, but as p gets larger, this method becomes wildly unreasonable.

Lucky for us, we have a way to check whether an integer is a quadratic residue with a single calculation thanks to Legendre. In the following, we will assume we are working modulo a prime $p$.

Before looking at Legendre's symbol, let's take a brief detour to see an interesting property of quadratic (non-)residues.

```
Quadratic Residue * Quadratic Residue = Quadratic Residue
Quadratic Residue * Quadratic Non-residue = Quadratic Non-residue
Quadratic Non-residue * Quadratic Non-residue = Quadratic Residue 
```
So what's the trick? The Legendre Symbol gives an efficient way to determine whether an integer is a quadratic residue modulo an odd prime p.

Legendre's Symbol: $(a / p) ≡ a^{(p-1)/2}$ mod p obeys:
```
(a / p) = 1 if a is a quadratic residue and a ≢ 0 mod p
(a / p) = -1 if a is a quadratic non-residue mod p
(a / p) = 0 if a ≡ 0 mod p 
```
Which means given any integer a, calculating `pow(a,(p-1)/2,p)` is enough to determine if a is a quadratic residue.

Now for the flag. Given the following 1024 bit prime and 10 integers, find the quadratic residue and then calculate its square root; the square root is your flag. Of the two possible roots, submit the larger one as your answer.

output.txt

In [22]:
with open("output_legendre.txt") as f: 
    lines = f.readlines()  

In [23]:
p=int(lines[0].strip('p =\n'))
p

101524035174539890485408575671085261788758965189060164484385690801466167356667036677932998889725476582421738788500738738503134356158197247473850273565349249573867251280253564698939768700489401960767007716413932851838937641880157263936985954881657889497583485535527613578457628399173971810541670838543309159139

In [24]:
int_list=[int(x) for x in lines[2].strip("ints =\n[]").split(',')]

In [17]:
quadr_res = [n for n in int_list if pow(n, (p-1)//2, p)==1][0]

Now that we have the quadratic residue, given that the prime is $p\equiv3\ mod\ 4$, we can find the square root. If $a$ is a quadratic residue, then according to [Euler's Criterion](https://en.wikipedia.org/wiki/Euler%27s_criterion)
$$a^{(p-1)/2}\equiv1\ mod\ p$$
It follows then that
$$a^{(p+1)/2}\equiv a^{(p-1)/2}a^{(p+1)/2}\equiv a^{\frac{(p-1)+(p+1)}{2}}\equiv a^p\equiv a\ mod\ p,$$ where the last equality follows from [Fermat's Little Theorem](https://en.wikipedia.org/wiki/Fermat%27s_little_theorem). Then since we know that $p\equiv 3 mod 4$, i.e. $p=4\cdot k+3$, it follows that $(p+1)/4$ is an integer. So, finally we have
$$(a^{(p+1)/4})^2\equiv a^{(p+1)/2}\equiv a\ mod\ p.$$ So, the aquare root is...

In [21]:
pow(quadr_res, (p+1)//4, p)

93291799125366706806545638475797430512104976066103610269938025709952247020061090804870186195285998727680200979853848718589126765742550855954805290253592144209552123062161458584575060939481368210688629862036958857604707468372384278049741369153506182660264876115428251983455344219194133033177700490981696141526

On to the next one!
## Modular Square Root
In Legendre Symbol we introduced a fast way to determine whether a number is a square root modulo a prime. We can go further: there are algorithms for efficiently calculating such roots. The best one in practice is called Tonelli-Shanks, which gets its funny name from the fact that it was first described by an Italian in the 19th century and rediscovered independently by Daniel Shanks in the 1970s.

All primes that aren't 2 are of the form `p ≡ 1 mod 4` or `p ≡ 3 mod 4`, since all odd numbers obey these congruences. As the previous challenge hinted, in the p ≡ 3 mod 4 case, a really simple formula for computing square roots can be derived directly from Fermat's little theorem. That leaves us still with the `p ≡ 1 mod 4` case, so a more general algorithm is required.

In a congruence of the form `r^2 ≡ a mod p`, Tonelli-Shanks calculates r.

The main use-case for this algorithm is finding elliptic curve co-ordinates. Its operation is somewhat complex so we're not going to discuss the details, however, implementations are easy to find and Sage has one built-in.

Find the square root of a modulo the 2048-bit prime p. Give the smaller of the two roots as your answer.

output.txt

In [25]:
with open("output_modular.txt") as f: 
    lines = f.readlines()

In [27]:
a=int(lines[0].strip('a =\n'))
p=int(lines[1].strip('p =\n'))

The following is an implementation of the Tonelli-Shanks theorem in python from [Svetlin Nakov](https://gist.github.com/nakov/60d62bdf4067ea72b7832ce9f71ae079)

In [29]:
def modular_sqrt(a, p):

    def legendre_symbol(a, p):
        """ Compute the Legendre symbol a|p using
            Euler's criterion. p is a prime, a is
            relatively prime to p (if p divides
            a, then a|p = 0)
            Returns 1 if a has a square root modulo
            p, -1 otherwise.
        """
        ls = pow(a, (p - 1) // 2, p)
        return -1 if ls == p - 1 else ls

    """ Find a quadratic residue (mod p) of 'a'. p
        must be an odd prime.
        Solve the congruence of the form:
            x^2 = a (mod p)
        And returns x. Note that p - x is also a root.
        0 is returned is no square root exists for
        these a and p.
        The Tonelli-Shanks algorithm is used (except
        for some simple cases in which the solution
        is known from an identity). This algorithm
        runs in polynomial time (unless the
        generalized Riemann hypothesis is false).
    """
    # Simple cases
    #
    if legendre_symbol(a, p) != 1:
        return 0
    elif a == 0:
        return 0
    elif p == 2:
        return p
    elif p % 4 == 3:
        return pow(a, (p + 1) // 4, p)

    # Partition p-1 to s * 2^e for an odd s (i.e.
    # reduce all the powers of 2 from p-1)
    #
    s = p - 1
    e = 0
    while s % 2 == 0:
        s //= 2
        e += 1

    # Find some 'n' with a legendre symbol n|p = -1.
    # Shouldn't take long.
    #
    n = 2
    while legendre_symbol(n, p) != -1:
        n += 1

    # Here be dragons!
    # Read the paper "Square roots from 1; 24, 51,
    # 10 to Dan Shanks" by Ezra Brown for more
    # information
    #

    # x is a guess of the square root that gets better
    # with each iteration.
    # b is the "fudge factor" - by how much we're off
    # with the guess. The invariant x^2 = ab (mod p)
    # is maintained throughout the loop.
    # g is used for successive powers of n to update
    # both a and b
    # r is the exponent - decreases with each update
    #
    x = pow(a, (s + 1) // 2, p)
    b = pow(a, s, p)
    g = pow(n, s, p)
    r = e

    while True:
        t = b
        m = 0
        for m in range(r):
            if t == 1:
                break
            t = pow(t, 2, p)

        if m == 0:
            return x

        gs = pow(g, 2 ** (r - m - 1), p)
        g = (gs * gs) % p
        x = (x * gs) % p
        b = (b * g) % p
        r = m

In [30]:
modular_sqrt(a, p)

2362339307683048638327773298580489298932137505520500388338271052053734747862351779647314176817953359071871560041125289919247146074907151612762640868199621186559522068338032600991311882224016021222672243139362180461232646732465848840425458257930887856583379600967761738596782877851318489355679822813155123045705285112099448146426755110160002515592418850432103641815811071548456284263507805589445073657565381850521367969675699760755310784623577076440037747681760302434924932113640061738777601194622244192758024180853916244427254065441962557282572849162772740798989647948645207349737457445440405057156897508368531939120

On to the next one!
## Chinese Remainder Theorem
The Chinese Remainder Theorem gives a unique solution to a set of linear congruences if their moduli are coprime.

This means, that given a set of arbitrary integers a<sub>i</sub>, and pairwise coprime integers n<sub>i</sub>, such that the following linear congruences hold:
```
x ≡ a1 mod n1
x ≡ a2 mod n2
...
x ≡ an mod nn
```

There is a unique solution `x ≡ a mod N` where `N = n1 * n2 * ... * nn`.

In cryptography, we commonly use the Chinese Remainder Theorem to help us reduce a problem of very large integers into a set of several, easier problems.

Given the following set of linear congruences:
```
x ≡ 2 mod 5
x ≡ 3 mod 11
x ≡ 5 mod 17 
```
Find the integer a such that `x ≡ a mod 935`

Starting with the congruence with the largest modulus, use that for `x ≡ a mod p` we can write `x = a + k*p` for arbitrary integer `k`.

In [40]:
from functools import reduce

def chinese_remainder(n, a):
    sum = 0
    prod = reduce(lambda a, b: a*b, n)
    for n_i, a_i in zip(n,a):
        p = prod/n_i
        sum += a_i * mul_inv(p, n_i) * p
    return sum % prod
    
def mul_inv(a, b):
    b0 = b
    x0, x1 = 0,1
    if b == 1:
        return 1
    while a > 1:
        q = a // b
        a , b = b, a%b
        x0, x1 = x1 -q*x0, x0
    if x1 < 0:
        x1 += b0
    return x1

a = [2,3,5] # x = a mod x
n = [5,11,17] # x = x mod n


print(chinese_remainder(n,a))

872.0


On to the next section!
# Lattices
## Vectors